# Unitary ansatz entering the VQE

The final energy output of a VQE calculation will crucially depend on the ansatz/form of the parameterized unitary $\hat U(\boldsymbol{\theta})$ employed in state preparation. Here we review two popular approaches, the unitary coupled cluster and qubit coupled cluster methodologies, and benchmark them for energy calculations of small molecules.

In [1]:
import numpy as np
import tequila as tq
from utility import *
threshold = 1e-6 #Cutoff for UCC MP2 amplitudes and QCC ranking gradients

## Unitary Coupled Cluster (UCC)

The UCC ansatz is obtained by 'unitarizing' the traditional coupled cluster ansatz,
$$ e^{\hat T} \rightarrow e^{\hat T - \hat T^\dagger} \equiv \hat U_{\text{UCC}}$$


Due to non-commutativity of terms in $\hat T - \hat T^\dagger$, the UCC ansatz does not have a straightforward decomposition in terms of circuit primitives implementable on the quantum computer. Therefore, to obtain a form which can be compiled, we employ the Trotter approximation. The accuracy of the circuit ansatz relative to the exact UCC operator will be dependent on how many Trotter steps are employed. The number of Trotter steps is commonly set to its minimal value of one to avoid excessive circuit depth.

In [2]:
trotter_steps = 1

### LiH in STO-3G basis

Below is a sample VQE simulation using the UCCSD ansatz compiled using a single trotter step for H$_2$ in minimal basis at $R=2.5$ (Angstrom). For comparison, we can run FCI to obtain the true ground state energy.


In [3]:
xyz_data = get_molecular_data('lih', geometry=2.5, xyz_format=True)
basis='sto-3g'

lih = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis)

print('Number of spin-orbitals (qubits): {} \n'.format(2*lih.n_orbitals))

E_FCI = lih.compute_energy(method='fci')

print('FCI energy: {}'.format(E_FCI))

Number of spin-orbitals (qubits): 12 

FCI energy: -7.823723883389364


The UCCSD VQE optimization is of the form
$$E = \min_{\boldsymbol{\theta}} \langle \text{HF} | \hat U_{\text{UCC}}^\dagger(\boldsymbol{\theta}) \hat H  \hat U_{\text{UCC}} (\boldsymbol{\theta}) | \text{HF} \rangle $$
To expedite the optimization process, we can set the initial guess of the amplitudes to zero, i.e. the optimization will begin at the Hartree Fock state. This heuristic is best suited for when Hartree Fock is believed to be in qualitative agreement with the true ground state. To further alleviate quantum resources, we can estimate the amplitudes using classical electronic structure methods (here, MP2 perturbation theory), and only include the unitaries with non-zero estimated amplitudes.

In [7]:
H = lih.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

U_UCCSD = lih.make_uccsd_ansatz(initial_amplitudes='MP2',threshold=threshold, trotter_steps=trotter_steps)

E = tq.ExpectationValue(H=H, U=U_UCCSD)

print('\nNumber of UCCSD amplitudes: {} \n'.format(len(E.extract_variables())))

print('\nStarting optimization:\n')

result = tq.minimize(objective=E, method="BFGS", initial_values={k:0.0 for k in E.extract_variables()}, tol=1e-6)

print('\nObtained UCCSD energy: {}'.format(result.energy))


Hamiltonian has 631 terms


Number of UCCSD amplitudes: 24 


Starting optimization:

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 1024 expectationvalues

active variables : 24

E=-7.77087366  angles= {(2, 1, 5, 0): 0.0, (5, 1, 2, 1): 0.0, (5, 0, 2, 0): 0.0, (2, 0, 2, 0): 0.0, (5, 0, 5, 0): 0.0, (4, 0, 4, 1): 0.0, (2, 1, 2, 1): 0.0, (2, 0, 5, 0): 0.0, (3, 1, 3, 1): 0.0, (4, 1, 4, 0): 0.0, (5, 1, 2, 0): 0.0, (3, 0, 3, 1): 0.0, (5, 0, 2, 1): 0.0, (5, 1, 5, 0): 0.0, (4, 0, 4, 0): 0.0, (2, 0, 5, 1): 0.0, (4, 1, 4, 1): 0.0, (2, 1, 5, 1): 0.0, (3, 0, 3, 0): 0.0, (5, 1, 5, 1): 0.0, (2, 0, 2, 1): 0.0, (5, 0, 5, 1): 0.0, (2, 1, 2, 0): 0.0, (3, 1, 3, 0): 0.0}  samples= None
E=-7.79445181  angles= {(2, 1, 5, 0): 0.0016832351684570312, (5, 1, 2, 1): -0.11050605773925781, (5, 0, 2, 0): -0.00811767578

E=-7.82110507  angles= {(2, 1, 5, 0): -0.0019150560470378832, (5, 1, 2, 1): -0.13789287768957362, (5, 0, 2, 0): -0.0005180307963037647, (2, 0, 2, 0): -0.0033117338325822617, (5, 0, 5, 0): -0.000925353948839398, (4, 0, 4, 1): 0.003875933171100165, (2, 1, 2, 1): -0.08633912794640079, (2, 0, 5, 0): -0.0005179655286454512, (3, 1, 3, 1): -0.01741256253764568, (4, 1, 4, 0): 0.0038760084004545708, (5, 1, 2, 0): -0.0019919675522675814, (3, 0, 3, 1): 0.003875995862861867, (5, 0, 2, 1): -0.0019140952550646322, (5, 1, 5, 0): 0.0014820881162258914, (4, 0, 4, 0): -0.0013316554767367026, (2, 0, 5, 1): -0.0019951792166694386, (4, 1, 4, 1): -0.01741888219814233, (2, 1, 5, 1): -0.1378912016791776, (3, 0, 3, 0): -0.0013319006078426991, (5, 1, 5, 1): -0.22363971041690206, (2, 0, 2, 1): 0.00031010470292761267, (5, 0, 5, 1): 0.0014821281461269192, (2, 1, 2, 0): 0.00031014361927858905, (3, 1, 3, 0): 0.0038761390430014525}  samples= None
E=-7.82111951  angles= {(2, 1, 5, 0): -0.0022517944120825886, (5, 1, 2,

E=-7.82120932  angles= {(2, 1, 5, 0): -0.0016721601877842723, (5, 1, 2, 1): -0.14243572913485095, (5, 0, 2, 0): -0.00035633450739939977, (2, 0, 2, 0): -0.0038338424991911733, (5, 0, 5, 0): -0.0012864523851314964, (4, 0, 4, 1): 0.003116046536271515, (2, 1, 2, 1): -0.09889263637418066, (2, 0, 5, 0): -0.0003555250664212455, (3, 1, 3, 1): -0.014497171619342629, (4, 1, 4, 0): 0.0031178541428295305, (5, 1, 2, 0): -0.0017743112098950046, (3, 0, 3, 1): 0.0031163292748252, (5, 0, 2, 1): -0.0016704322128881924, (5, 1, 5, 0): 0.0009025018262581702, (4, 0, 4, 0): -0.0018270474785841732, (2, 0, 5, 1): -0.0017767224395119972, (4, 1, 4, 1): -0.014499030011895639, (2, 1, 5, 1): -0.14242668362799613, (3, 0, 3, 0): -0.001828929354072269, (5, 1, 5, 1): -0.2194810173726105, (2, 0, 2, 1): -0.0004669880392421378, (5, 0, 5, 1): 0.0009032765916138759, (2, 1, 2, 0): -0.0004673424946210798, (3, 1, 3, 0): 0.0031172383185807035}  samples= None
E=-7.82120930  angles= {(2, 1, 5, 0): -0.0017313147359400544, (5, 1, 2

E=-7.82120933  angles= {(2, 1, 5, 0): -0.001684454276735427, (5, 1, 2, 1): -0.14242311265840504, (5, 0, 2, 0): -0.0003509041436890166, (2, 0, 2, 0): -0.0038372346003395464, (5, 0, 5, 0): -0.001289814959141693, (4, 0, 4, 1): 0.00310495635415608, (2, 1, 2, 1): -0.09892884077110149, (2, 0, 5, 0): -0.0003496637670074464, (3, 1, 3, 1): -0.014498463662495775, (4, 1, 4, 0): 0.003107110532722815, (5, 1, 2, 0): -0.0017861540507342335, (3, 0, 3, 1): 0.003106460756803581, (5, 0, 2, 1): -0.0016835625746498281, (5, 1, 5, 0): 0.0009184371895083866, (4, 0, 4, 0): -0.001823014075398057, (2, 0, 5, 1): -0.0017884485771910712, (4, 1, 4, 1): -0.014499564598494481, (2, 1, 5, 1): -0.14240858064269005, (3, 0, 3, 0): -0.0018230911678993964, (5, 1, 5, 1): -0.21947788645332592, (2, 0, 2, 1): -0.0004620914103864106, (5, 0, 5, 1): 0.0009193471535329275, (2, 1, 2, 0): -0.00046218844538350827, (3, 1, 3, 0): 0.0031069879012936524}  samples= None
E=-7.82120933  angles= {(2, 1, 5, 0): -0.001684398023786038, (5, 1, 2, 

We see that the converged UCCSD energy is in exact agreement with the FCI energy, as expected for a $2$-electron system. 

## Qubit Coupled Cluster (QCC)

In contrast to UCC, the QCC methodology makes no direct reference to fermionic algebra and seeks to construct an efficient ansatz directly in qubit-space by finding multi-qubit Pauli strings (entanglers) which lower energy. This is done through an energy-lowering heuristic employing the energy gradient with respect to a Pauli strings variational amplitude. As opposed to UCCSD, the circuit depth and number of parameter is chosen to meet hardware limitations, i.e. one must choose how many exponentiated Pauli strings will be entering the QCC ansatz.

### LiH in STO-3G basis

Below we perform the entangler screening protocol for LiH in minimal basis, and obtain one grouping of entanglers with non-zero energy gradient. We then select one of them to be used in the QCC VQE simulation.

In [8]:
xyz_data = get_molecular_data('lih', geometry=2.5, xyz_format=True)
basis='sto-3g'

lih = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set='sto-3g')

hf_reference = hf_occ(2*lih.n_orbitals, lih.n_electrons)

H = lih.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

#Define number of entanglers to enter ansatz
n_ents = 1

#Rank entanglers using energy gradient criterion
ranked_entangler_groupings = generate_QCC_gradient_groupings(H.to_openfermion(), 
                                                             2*lih.n_orbitals, 
                                                             hf_reference, 
                                                             cutoff=threshold)

print('Grouping gradient magnitudes (Grouping : Gradient magnitude):')
for i in range(len(ranked_entangler_groupings)):
    print('{} : {}'.format(i+1,ranked_entangler_groupings[i][1]))


entanglers = get_QCC_entanglers(ranked_entangler_groupings, n_ents, 2*lih.n_orbitals)

print('\nSelected entanglers:')
for ent in entanglers:
    print(ent)




Hamiltonian has 631 terms

Grouping gradient magnitudes (Grouping : Gradient magnitude):
1 : 0.1321
2 : 0.0553
3 : 0.0553
4 : 0.0323
5 : 0.022
6 : 0.0208
7 : 0.0208
8 : 0.0102
9 : 0.0098
10 : 0.0098
11 : 0.0074
12 : 0.0074
13 : 0.0074
14 : 0.0074
15 : 0.0049
16 : 0.0049
17 : 0.0041
18 : 0.0041
19 : 0.003
20 : 0.003
21 : 0.002
22 : 0.002
23 : 0.0019
24 : 0.0019
25 : 0.0005
26 : 0.0005
27 : 0.0
28 : 0.0
29 : 0.0
30 : 0.0
31 : 0.0
32 : 0.0
33 : 0.0
34 : 0.0

Selected entanglers:
1.0 [X2 Y3 X10 X11]


Once the QCC ranking procedure has been ran, we can simulate the QCC VQE optimization with the generated entanglers. The VQE optimization for the QCC ansatz is of the form
$$E = \min_{\boldsymbol{\Omega}, \boldsymbol{\tau}} \langle \boldsymbol{\Omega} | U_{\text{ENT}}^\dagger (\boldsymbol{\tau}) \hat H  U_{\text{ENT}} (\boldsymbol{\tau}) | \boldsymbol{\Omega} \rangle $$
where $\boldsymbol{\Omega}$ denote collective Euler angles parameterizing single-qubit rotations, and $\boldsymbol{\tau}$ are entangler amplitudes. 

In [9]:
#Mean-field part of U (Omega):    
U_MF = construct_QMF_ansatz(n_qubits = 2*lih.n_orbitals)
#Entangling part of U:
U_ENT = construct_QCC_ansatz(entanglers)

U_QCC = U_MF + U_ENT

E = tq.ExpectationValue(H=H, U=U_QCC)

initial_vals = init_qcc_params(hf_reference, E.extract_variables())

#Minimize wrt the entangler amplitude and MF angles:
result = tq.minimize(objective=E, method="BFGS", initial_values=initial_vals, tol=1.e-6)

print('\nObtained QCC energy ({} entanglers): {}'.format(len(entanglers), result.energy))

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 50 expectationvalues

active variables : 25

E=-6.75505059  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 3.141592653589793, gamma_2: 0.0, beta_3: 3.141592653589793, gamma_3: 0.0, beta_4: 0.0, gamma_4: 0.0, beta_5: 0.0, gamma_5: 0.0, beta_6: 0.0, gamma_6: 0.0, beta_7: 0.0, gamma_7: 0.0, beta_8: 0.0, gamma_8: 0.0, beta_9: 0.0, gamma_9: 0.0, beta_10: 3.141592653589793, gamma_10: 0.0, beta_11: 3.141592653589793, gamma_11: 0.0, tau_0: 0.0}  samples= None
Optimization terminated successfully.
         Current function value: -6.755051
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1

Obtained QCC energy (1 entanglers): -6.7550505907559755


We see that the QCC energy converged to the FCI energy with only a single entangler! 